In [102]:
freq_dict = {'15-min':'15T',
            '1-hour':'1H',
            '2-hour':'2H',
            '4-hour':'4H',
            '6-hour':'6H',
            '8-hour':'8H',
            '12-hour':'12H',
            '1-day':'1D',
            '2-day':'2D',
            '7-day':'7D',
            '15-day':'15D',
            '1-month':'1M'}

In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from copy import deepcopy



In [104]:
metadata_df = pd.read_csv("../metadata/metadata.csv",index_col=0)

In [136]:
city_count = {}
for i in metadata_df.index:
    city = metadata_df.loc[i]['city']
    if city in city_count.keys():
        city_count[city] += 1
    else:
        city_count[city] = 1

In [137]:
city = 'Austin'

In [152]:
st = pd.HDFStore(os.path.expanduser("/Users/xox/monthly-12-17.h5"), mode='r')

In [153]:
freq='1M'

In [154]:
city_data = metadata_df[metadata_df['city'] == city]
city_homes = city_data.index.values.astype('int')

In [155]:
count

82

In [156]:
homes = {}
count = 0
for freq in ['1M']:
    print(freq)
    freq_num = int(freq[:-1])
    out = []
    homes[freq] = []
    city_data = metadata_df[metadata_df['city'] == city]
    city_homes = city_data.index.values.astype('int')
    for home in city_homes:
#         print(home)
        try: 
            
            data_df = st['/%d' %home]
            count += 1
#             print(count)
            homes[home] = data_df.resample(freq).mean()
        except Exception as e:
            continue


1M


In [151]:
count

82

In [112]:
home_ids = homes.keys()

In [113]:
home_ids

dict_keys([9451, 645, 8198, 7693, 1854, 8597, 22, 6423, 5403, 5539, 4438, 5673, 6061, 6578, 9139, 575, 3126, 6072, 6460, 6078, 2751, 4800, 7491, 3143, 457, 7504, 6614, 6871, 4699, 8029, 222, 4575, 5456, 483, 3044, 5994, 491, 5356, 4590, 8829, 624, 6643, 3134, 4473, 5439, 6012, '1M'])

In [121]:
home_readings = {}
for year in range(2012, 2019):
    home_readings[year] = {}
for hid in home_ids:
    for year in range(2012, 2019):
        try:
            year_readings = homes[hid][str(year)]
            if len(year_readings) < 12:
                continue
            for m in range(12):
                old_index = year_readings.index[m]
                new_index = str(old_index)[:7]
                year_readings = year_readings.rename(index={old_index:new_index})
                home_readings[year][hid] = year_readings
                
        except:
            continue

In [125]:
len(home_readings[2015].keys())

18

In [126]:
# with all observed aggregate readings
all_aggregate = {}
for year in range(2013, 2019):
    all_aggregate[year] = {}
    for hid in home_readings[year].keys():
        try:
            if home_readings[year][hid]['use'].isnull().sum() == 0:
                all_aggregate[year][hid] = home_readings[year][hid]
        except Exception as e:
            continue
for year in range(2013, 2019):
    print(year, len(all_aggregate[year].keys()))

2013 9
2014 25
2015 17
2016 15
2017 9
2018 7


In [127]:
year = 2015

In [128]:
app_count = {}
for year in range(2013, 2019):
    app_count[year] = {}
    for hid in all_aggregate[year].keys():
        app_nan = all_aggregate[year][hid].isnull().sum()
        for app in app_nan.keys():
            if app_nan[app] == 0:
                if app in app_count[year]:
                    app_count[year][app] += 1
                else:
                    app_count[year][app] = 1

In [129]:
df = pd.DataFrame(app_count)

In [130]:
app_count

{2013: {'air1': 3,
  'bedroom1': 3,
  'clotheswasher1': 2,
  'diningroom1': 1,
  'dishwasher1': 3,
  'disposal1': 1,
  'drye1': 1,
  'furnace1': 3,
  'kitchenapp1': 1,
  'kitchenapp2': 1,
  'lights_plugs1': 3,
  'lights_plugs2': 1,
  'lights_plugs4': 1,
  'lights_plugs5': 1,
  'oven1': 1,
  'range1': 2,
  'refrigerator1': 3,
  'use': 9},
 2014: {'air1': 16,
  'bedroom1': 4,
  'bedroom2': 1,
  'car1': 2,
  'clotheswasher1': 4,
  'dishwasher1': 17,
  'disposal1': 9,
  'drye1': 8,
  'dryg1': 1,
  'furnace1': 17,
  'kitchen1': 6,
  'kitchen2': 3,
  'kitchenapp1': 7,
  'kitchenapp2': 5,
  'lights_plugs1': 3,
  'lights_plugs2': 1,
  'lights_plugs4': 1,
  'lights_plugs5': 1,
  'livingroom1': 8,
  'microwave1': 7,
  'oven1': 1,
  'poolpump1': 1,
  'range1': 11,
  'refrigerator1': 16,
  'use': 25,
  'waterheater1': 8},
 2015: {'air1': 14,
  'air2': 1,
  'bathroom1': 1,
  'bedroom1': 4,
  'bedroom2': 2,
  'car1': 3,
  'clotheswasher1': 5,
  'clotheswasher_dryg1': 1,
  'diningroom1': 1,
  'dishwa

In [131]:
df[(df > 100).any(1)]

,2013,2014,2015,2016,2017,2018


In [132]:
candidate_app = df[(df > 30).all(1)].index.tolist()

In [133]:
candidate_app = ['air1', 'clotheswasher1', 'dishwasher1', 'drye1', 'furnace1', 'kitchenapp1', 'microwave1', 'refrigerator1', 'use']

In [134]:
c = 0
overall = 0
cand = {}
for year in range(2013, 2019):
    cand[year] = {}
    for hid in all_aggregate[year].keys():
        if all_aggregate[year][hid].isnull().sum()[candidate_app].sum() == 0:
#             print(year, hid)
            df = pd.DataFrame(all_aggregate[year][hid])
            candidate = df.iloc[:, df.columns.isin(candidate_app)]
            if len(candidate.columns) == len(candidate_app):
                cand[year][hid] = candidate
    print(year, len(cand[year].keys()))
    overall += len(cand[year].keys())
print('overall', overall)

2013 0
2014 1
2015 0
2016 0
2017 0
2018 0
overall 1


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/core/series.py:705: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [82]:
def get_dataset(selected_years, selected_appliance, all_observed, num_columns=1):
    data = {}
    for year in selected_years:
        data[year] = {}
        for hid in all_aggregate[year].keys():
            df = pd.DataFrame(all_aggregate[year][hid])
            candidate = df.iloc[:, df.columns.isin(selected_appliance)]
            if all_observed:
                if len(candidate.columns) == len(selected_appliance) and candidate.isnull().sum().sum() == 0:
                    data[year][hid] = candidate
            else:
                if len(candidate.columns) >= num_columns:
                    data[year][hid] = candidate

    return data
            

In [83]:
def get_info(data):
    overall = 0
    for year in data.keys():
        print(year, len(data[year].keys()))
        overall += len(data[year].keys())
    print("overall: ", overall)

In [84]:
selected_years = [2013, 2014, 2015, 2016, 2017, 2018]
selected_appliance = ['air1', 'clotheswasher1', 'dishwasher1', 'furnace1',  'microwave1', 'refrigerator1', 'use']
all_observed = True

In [85]:
data = get_dataset(selected_years, selected_appliance, all_observed, 2)
get_info(data)

2016 1
2017 2
2018 0
2013 0
2014 0
2015 2
overall:  5


In [503]:
np.save("./data-2013-2017-observed.npy", data)